In [4]:
import configparser
from pathlib import Path
import os
from psycopg2 import sql
import psycopg2

In [2]:
path = Path(__vsc_ipynb_file__)
ROOT_DIR = path.parent.parent.absolute()
config_path = os.path.join(ROOT_DIR, "config.conf")

config = configparser.ConfigParser()
config.read(config_path)

['c:\\Users\\hares\\singaporesub\\config.conf']

In [3]:
USERNAME = config.get("redshift_config", "redshift_username")
PASSWORD = config.get("redshift_config", "redshift_password")
HOST = config.get("redshift_config", "redshift_hostname")
PORT = config.get("redshift_config", "redshift_port")
DATABASE = config.get("redshift_config", "redshift_database")


In [55]:
sql_create_comments_fact_table = sql.SQL("""
CREATE TABLE comments_fact (
"comment_id" VARCHAR(7) NOT NULL,
"post_id" VARCHAR(6) NOT NULL,
"author_id" VARCHAR(10),
"body" VARCHAR(700),
"created_date" TIMESTAMPTZ ,
"score" INT,
"edited" BOOLEAN default False,
"compound" DECIMAL(5,4),
"comp_score" VARCHAR(4)
)

""")

In [56]:
sql_load_comments = sql.SQL("""
   COPY comments_fact (comment_id,post_id,author_id,body,created_date,score,edited,compound,comp_score) 
   FROM 's3://comments-transformed-bucket/transformed_comments.csv'  credentials 'aws_iam_role=arn:aws:iam::075207432376:role/REDHSIFTACESS'
FORMAT AS CSV DELIMITER ',' QUOTE '"'
region 'ap-southeast-1'

 IGNOREHEADER 1                   
       
                            
                            
                            
                            
                            """)

In [57]:
def connect_to_redshift():
    try:
        rs_wh = psycopg2.connect(dbname = DATABASE , user = USERNAME , password = PASSWORD , host = HOST , port = PORT)
        rs_wh.autocommit = True

        return rs_wh
    except Exception as e:
        print(f"Unable to connect to Redshift. Error {e}")
        sys.exit(1)

In [59]:
def main():
    rs_conn = connect_to_redshift().cursor()
    rs_conn.execute(sql_create_comments_fact_table)
    rs_conn.execute(sql_load_comments)

In [60]:
if __name__ == "__main__":
    main()

InternalError_: Load into table 'comments_fact' failed.  Check 'stl_load_errors' system table for details.


In [ ]:
``